In [2]:
import os
import wandb
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ANANYA\_netrc
wandb: Currently logged in as: anu-mh0206 (anu-mh0206-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
# Get the dataset from UCI
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data -qq

# modified from https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py
# Import wandb
import wandb
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score

run = wandb.init(project="Lab1-visualize-models", name="xgboost-tuned")

# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.2
param['max_depth'] = 4
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5

# Add the wandb xgboost callback
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.WandbCallback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

run.summary['Error Rate'] = error_rate
accuracy = accuracy_score(test_Y, pred)
wandb.log({"accuracy": accuracy})
wandb.sklearn.plot_confusion_matrix(test_Y, pred, [0., 1., 2., 3., 4., 5.])

run.finish()

[0]	train-mlogloss:1.29920	test-mlogloss:1.37204
[1]	train-mlogloss:1.01319	test-mlogloss:1.10188
[2]	train-mlogloss:0.81308	test-mlogloss:0.91946
[3]	train-mlogloss:0.66429	test-mlogloss:0.78956
[4]	train-mlogloss:0.54813	test-mlogloss:0.68650


D:\MLOps_Coursework\MLOps-course\mlops-course\Lib\site-packages\wandb\integration\xgboost\xgboost.py:117: UserWarning: [01:39:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "silent" } are not used.

  config = model.save_config()


Test error using softmax = 0.08181818181818182


accuracy,▁
epoch,▁▃▅▆█
test-mlogloss,█▅▃▂▁
train-mlogloss,█▅▃▂▁
Error Rate,0.08182
accuracy,0.91818
epoch,4
